In [1]:
#### loading the trained siamese network and checking its accuracy on the test data kept aside.
#### taking the polygon location from test data itself instead of running the image through 
#### spacenet model to get polgon predictions. This should give better idea about how well siamese network has trained


In [2]:
import sys
import numpy as np
import pandas as pd
# from scipy.misc import imread
import pickle
import os
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import time

import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from PIL import Image

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K

from tqdm import tqdm

from sklearn.utils import shuffle

import numpy as np
import shapely.wkt
import shapely
from shapely.geometry import Polygon

/home/abhirag/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/abhirag/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/abhirag/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/abhirag/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

In [3]:
from keras.models import load_model

model = load_model('siamese_classification.h5')




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [4]:
import json, os, random
from PIL import Image

def process_img(img_array, polygon_pts):
    height, width, _ = img_array.shape

    xcoords = polygon_pts[:, 0]
    ycoords = polygon_pts[:, 1]
    xmin, xmax = np.min(xcoords), np.max(xcoords)
    ymin, ymax = np.min(ycoords), np.max(ycoords)

    xdiff = xmax - xmin
    ydiff = ymax - ymin

    return img_array[int(ymin):int(ymax), int(xmin):int(xmax), :]

# def process_img(img_array, polygon_pts, scale_pct):
#     height, width, _ = img_array.shape

#     xcoords = polygon_pts[:, 0]
#     ycoords = polygon_pts[:, 1]
#     xmin, xmax = np.min(xcoords), np.max(xcoords)
#     ymin, ymax = np.min(ycoords), np.max(ycoords)

#     xdiff = xmax - xmin
#     ydiff = ymax - ymin

#     #Extend image by scale percentage
# #     xmin = max(int(xmin - (xdiff * scale_pct)), 0)
# #     xmax = min(int(xmax + (xdiff * scale_pct)), width)
# #     ymin = max(int(ymin - (ydiff * scale_pct)), 0)
# #     ymax = min(int(ymax + (ydiff * scale_pct)), height)

#     return img_array[int(ymin):int(ymax), int(xmin):int(xmax), :]

base = '/home/abhirag/ONLY_PORTUGAL/validation/'

images_list = [base + i for i in os.listdir(base)]

random.shuffle(images_list)

In [5]:




predicted_labels = []
actual_labels = []
################################################################################################
for i in images_list:
    
#     i = '/home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001119_post_disaster.png'
    
    if i.split('_')[3] == 'post' and i.split('.')[-1] == 'png':

        
        
        print('PROCESSING +++++++ ', i)
        post_image_to_process = i
        pre_image_to_process = i.replace('_post_', '_pre_')

        pre_img_arr = np.array(Image.open(pre_image_to_process))
        post_img_arr = np.array(Image.open(post_image_to_process))


        post_json = json.load(open(post_image_to_process.replace('png', 'json')))
        pre_json = json.load(open(pre_image_to_process.replace('png', 'json')))
        

        
        for j in range(len(post_json['features']['xy'])):
            
            if post_json['features']['xy'][j]['properties']['subtype'] != 'un-classified':
                aa = []

                #polygon_original = shapely.wkt.loads(post_json['features']['xy'][j]['wkt'])
                poly = np.array(list(shapely.wkt.loads(post_json['features']['xy'][j]['wkt']).exterior.coords))
                aa.append(np.array(Image.fromarray(process_img(post_img_arr, poly)).resize((224,224))).reshape(1,224,224,3))

                uid = post_json['features']['xy'][j]['properties']['uid']

                for k in range(len(pre_json['features']['xy'])):
                    if pre_json['features']['xy'][k]['properties']['uid'] == uid :
                        poly_pre = np.array(list(shapely.wkt.loads(pre_json['features']['xy'][k]['wkt']).exterior.coords))
                        #print(Image.fromarray(process_img(pre_img_arr, poly_pre)))
                        #aa.append(np.array(process_img(pre_img_arr, poly_pre)).reshape(1,224,224,3))
                        aa.append(np.array(Image.fromarray(process_img(pre_img_arr, poly_pre)).resize((224,224))).reshape(1,224,224,3))


                actual_labels.append(post_json['features']['xy'][j]['properties']['subtype'])



                predicted_labels.append(model.predict(aa))

        


PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001059_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000391_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001751_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000603_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001104_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000107_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000117_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000174_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000203_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portu

PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000165_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001453_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000087_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000167_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000962_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000379_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001132_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001726_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001387_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portu

PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001124_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000338_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000933_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000381_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000318_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000799_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000426_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000036_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000687_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portu

PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001544_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001802_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001568_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000485_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000526_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001678_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001541_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000424_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001163_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portu

PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000860_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001256_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001499_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001534_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000274_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001323_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00001759_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000182_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portugal-wildfire_00000854_post_disaster.png
PROCESSING +++++++  /home/abhirag/ONLY_PORTUGAL/validation/portu

In [6]:
predicted_labels

[array([[0.74319637]], dtype=float32),
 array([[0.856301]], dtype=float32),
 array([[0.9843599]], dtype=float32),
 array([[0.44874033]], dtype=float32),
 array([[0.36567628]], dtype=float32),
 array([[0.9680089]], dtype=float32),
 array([[0.48508513]], dtype=float32),
 array([[0.6383219]], dtype=float32),
 array([[0.84009624]], dtype=float32),
 array([[0.28536886]], dtype=float32),
 array([[0.9190908]], dtype=float32),
 array([[0.95736575]], dtype=float32),
 array([[0.8595314]], dtype=float32),
 array([[0.30926058]], dtype=float32),
 array([[0.3455338]], dtype=float32),
 array([[0.6347808]], dtype=float32),
 array([[0.5797268]], dtype=float32),
 array([[0.7759127]], dtype=float32),
 array([[0.79676]], dtype=float32),
 array([[0.837781]], dtype=float32),
 array([[0.72471786]], dtype=float32),
 array([[0.2170883]], dtype=float32),
 array([[0.21267086]], dtype=float32),
 array([[0.24325763]], dtype=float32),
 array([[0.88271815]], dtype=float32),
 array([[0.35213116]], dtype=float32),
 ar

In [13]:
pred_4_class = []
for pred in predicted_labels :
    if pred < 0.5 :
        pred_4_class.append('no-damage')
    elif pred < 0.7 :
        pred_4_class.append('minor-damage')
    elif pred < 0.9 :
        pred_4_class.append('major-damage')
    else :
        pred_4_class.append('destroyed')

In [14]:
from sklearn.metrics import classification_report

print(classification_report(y_true = actual_labels, y_pred = pred_4_class))

              precision    recall  f1-score   support

   destroyed       0.21      0.66      0.32       163
major-damage       0.02      0.28      0.04        65
minor-damage       0.01      0.25      0.02        32
   no-damage       0.99      0.50      0.67      3555

    accuracy                           0.50      3815
   macro avg       0.31      0.42      0.26      3815
weighted avg       0.93      0.50      0.64      3815



In [9]:
np.unique(actual_labels)


array(['destroyed', 'major-damage', 'minor-damage', 'no-damage'],
      dtype='<U12')